# 1. Set up environment:

In [ ]:
"""! git clone https://github.com/huggingface/transformers
%cd transformers
! pip install --quiet .

# Install the vncorenlp python wrapper
! pip install --quiet vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)
%mkdir -p vncorenlp/models/wordsegmenter
! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
%mv VnCoreNLP-1.1.1.jar vncorenlp/
%mv vi-vocab vncorenlp/models/wordsegmenter/
%mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

! wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
! tar -xzvf PhoBERT_base_fairseq.tar.gz

! wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
! tar -xzvf PhoBERT_base_transformers.tar.gz
"""

'! git clone https://github.com/huggingface/transformers\n%cd transformers\n! pip install --quiet .\n\n# Install the vncorenlp python wrapper\n! pip install --quiet vncorenlp\n\n# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)\n%mkdir -p vncorenlp/models/wordsegmenter\n! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar\n! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab\n! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr\n%mv VnCoreNLP-1.1.1.jar vncorenlp/\n%mv vi-vocab vncorenlp/models/wordsegmenter/\n%mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/\n\n! wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz\n! tar -xzvf PhoBERT_base_fairseq.tar.gz\n\n! wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz\n! tar -xzvf PhoBERT_base_transformers.tar.gz\n\n!pip install --quiet fastBPE\n!pip install 

In [ ]:
%cd /content/drive/MyDrive/transformers
! pip install --quiet .

/content/drive/MyDrive/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.2 MB/s eta 0:00:00


In [ ]:
%cd /content/
!pip install --quiet fastBPE
!pip install --quiet fairseq
! pip install --quiet vncorenlp

/content
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 62.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from vncorenlp import VnCoreNLP
import pandas as pd

from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

# 2. Feature engineering

In [ ]:
# Load tokenizer, Byte Pair Encoder function, and vocabulary available in PhoBERT
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/transformers/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes',
    default="/content/drive/MyDrive/transformers/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

vocab = Dictionary()
vocab.add_from_file("/content/drive/MyDrive/transformers/PhoBERT_base_transformers/dict.txt")

### 2.1 Train, validation, test set split (70, 10, 20)

In [ ]:
import re
from sklearn.model_selection import train_test_split

data = pd.read_excel("/content/drive/MyDrive/THESIS DSEB62: Product review analysis/Data/processed.xlsx")

X_train, X_test, y_train, y_test = train_test_split(data["comment"], data[["Quality", "Pack", "Serve", "Shipping"]], test_size= 0.2, random_state= 60)


train_id, train_text, train_labels = [], [], []
test_id, test_text = [], []

for text in X_train:
    text = rdrsegmenter.tokenize(text)
    text = ' '.join([' '.join(x) for x in text])
    train_text.append(text)


for text in X_test:
    text = rdrsegmenter.tokenize(text)
    text = ' '.join([' '.join(x) for x in text])
    test_text.append(text)



In [ ]:
current_label = "Quality"
train_labels = list(y_train[current_label])
test_labels = list(y_test[current_label])

In [ ]:
train_sents, val_sents, train_labels, val_labels = train_test_split(train_text, train_labels, test_size=0.1)

## 2.2. Byte Pair Encoding
(Map the words to their indexes in the vocabulary)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 125

train_ids = []
for sent in train_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_sent)

val_ids = []
for sent in val_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    val_ids.append(encoded_sent)

test_ids = []
for sent in test_text:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    test_ids.append(encoded_sent)

# Pad 0 at the end of the sequence to make its length fixed at 125 words
test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

## 2.3. Create masks
(Replace the non-zero components in BPE steps with 1)

In [ ]:
train_masks = []
for sent in train_ids:
    mask = [int(token_id > 0) for token_id in sent]
    train_masks.append(mask)

val_masks = []
for sent in val_ids:
    mask = [int(token_id > 0) for token_id in sent]
    val_masks.append(mask)

test_masks = []
for sent in test_ids:
    mask = [int(token_id > 0) for token_id in sent]
    test_masks.append(mask)

In [ ]:
print("Original sentence:")
print(train_sents[1])
print("----")
print("Padded Byte Pair Encoding:")
print(train_ids[1])
print("----")
print("Masked encoding:")
print(train_masks[1])

Original sentence:
đồng_hồ không kết_nối được với điện_thoại
----
Padded Byte Pair Encoding
[   0 1935   17 1127   11   15  601    2    2    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]
----
Masked encoding
tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0

## 2.4. Convert to TensorDataset to work with Pytorch

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)
test_inputs = torch.tensor(test_ids)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
test_masks = torch.tensor(test_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=1)

# 3. Modelling

# 3.1 Load pre-trained model

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

# Load model PhoBERT base (smaller than Large)
config = RobertaConfig.from_pretrained(
    "/content/drive/MyDrive/transformers/PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 2, output_hidden_states=False,
)
PHO_BERT = RobertaForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/transformers/PhoBERT_base_transformers/model.bin",
    config=config
)

# Transfer the model to GPU and print model architecure
PHO_BERT.cuda()

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/transformers/PhoBERT_base_transformers/model.bin and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=0)
      (position_embeddings): Embedding(258, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

# define evaluation metrics (f1-score and accuracy)
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    F1_score = f1_score(pred_flat, labels_flat, average='macro')

    return accuracy_score(pred_flat, labels_flat), F1_score

In [ ]:
import random
from tqdm import tqdm_notebook
device = 'cuda'
epochs = 10

param_optimizer = list(PHO_BERT.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)
# Save model with best accuracy on validation set
save_best_path =  "/content/drive/MyDrive/THESIS DSEB62: Product review analysis/Baseline-model/Phobert result/Weights/"+ current_label + "/best.pth"
# Save model after 10 epochs
save_last_path = "/content/drive/MyDrive/THESIS DSEB62: Product review analysis/Baseline-model/Phobert result/Weights/"+ current_label + "/last.pth"
max_acc = 0
best_epoch = 0
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    PHO_BERT.train()
    train_accuracy = 0
    nb_train_steps = 0
    train_f1 = 0

    for step, batch in tqdm_notebook(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        PHO_BERT.zero_grad()
        outputs = PHO_BERT(b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()

        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_train_accuracy, tmp_train_f1 = flat_accuracy(logits, label_ids)
        train_accuracy += tmp_train_accuracy
        train_f1 += tmp_train_f1
        nb_train_steps += 1

        loss.backward()
        torch.nn.utils.clip_grad_norm_(PHO_BERT.parameters(), 1.0)
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(" Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
    print(" F1 score: {0:.4f}".format(train_f1/nb_train_steps))
    print(" Average training loss: {0:.4f}".format(avg_train_loss))

    print("Running Validation...")
    PHO_BERT.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1 = 0
    for batch in tqdm_notebook(val_dataloader):

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = PHO_BERT(b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)

            eval_accuracy += tmp_eval_accuracy
            eval_f1 += tmp_eval_f1
            nb_eval_steps += 1
    print(" Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print(" F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))
    if (eval_accuracy/nb_eval_steps) > max_acc:
        print("new model saved")
        max_acc = eval_accuracy/nb_eval_steps
        torch.save(PHO_BERT, save_best_path)
        best_epoch = epoch_i
print("Training complete!")
torch.save(PHO_BERT, save_last_path)

======== Epoch 1 / 10 ========
Training...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-83-d62cc0657c17>:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in tqdm_notebook(enumerate(train_dataloader)):


0it [00:00, ?it/s]

 Accuracy: 0.8370
 F1 score: 0.6221
 Average training loss: 0.3953
Running Validation...


<ipython-input-83-d62cc0657c17>:62: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(val_dataloader):


...

0it [00:00, ?it/s]

 Accuracy: 0.9711
 F1 score: 0.9509
 Average training loss: 0.0944
Running Validation...


<ipython-input-83-d62cc0657c17>:62: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(val_dataloader):


  0%|          | 0/39 [00:00<?, ?it/s]

 Accuracy: 0.8341
 F1 score: 0.7053
Training complete!


In [ ]:
# Prepare test set same steps as train set
test_ids = []

for sent in test_text:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    test_ids.append(encoded_sent)

test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [ ]:
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)

<ipython-input-85-81cf86713975>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels = torch.tensor(test_labels)
<ipython-input-85-81cf86713975>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_masks = torch.tensor(test_masks)


In [ ]:
PHO_BERT = torch.load(f"/content/drive/MyDrive/THESIS DSEB62: Product review analysis/Baseline-model/Phobert result/Weights/{current_label}/best.pth")
with torch.no_grad():
    predictions = []
    labels_l = []
    correct = 0
    total = 0
    for step, batch in tqdm_notebook(enumerate(test_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        outputs = PHO_BERT(b_input_ids,
        token_type_ids=None,
        attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy().item()
        preds = np.argmax(logits, axis=1)[0]
        labels_l.append(label_ids)
        predictions.append(preds)


<ipython-input-86-1cf1d9908aa6>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in tqdm_notebook(enumerate(test_dataloader)):


0it [00:00, ?it/s]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions, labels_l))

              precision    recall  f1-score   support

           0       0.38      0.77      0.51       289
           1       0.97      0.87      0.92      2798

    accuracy                           0.86      3087
   macro avg       0.68      0.82      0.71      3087
weighted avg       0.92      0.86      0.88      3087



In [ ]:
# Save predictions
df = pd.read_excel("/content/drive/MyDrive/THESIS DSEB62: Product review analysis/Baseline-model/Phobert result/Phobert_pred.xlsx")
df[current_label] = predictions
df.to_excel("/content/drive/MyDrive/THESIS DSEB62: Product review analysis/Baseline-model/Phobert result/Phobert_pred.xlsx", encoding = "utf-16")

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 4. Evaluate model

In [ ]:
from sklearn.metrics import hamming_loss
from sklearn.metrics import classification_report

phobert_pred = pd.read_excel("/content/drive/MyDrive/THESIS DSEB62: Product review analysis/Baseline-model/Phobert result/Phobert_pred.xlsx")

label_cols = ["Quality", "Pack", "Serve", "Shipping"]
print("Classification report from phobert")
print('Hamming Loss: ', round(hamming_loss(y_test[["Quality", "Pack", "Serve", "Shipping"]], phobert_pred[["Quality", "Pack", "Serve", "Shipping"]]),2))
for i in range(len(label_cols)):
  print(f"classification report of {label_cols[i]}")
  print(classification_report(y_test[label_cols[i]], phobert_pred[label_cols[i]]))

Classification report from phobert
Hamming Loss:  0.08
classification report of Quality
              precision    recall  f1-score   support

           0       0.77      0.38      0.51       586
           1       0.87      0.97      0.92      2501

    accuracy                           0.86      3087
   macro avg       0.82      0.68      0.71      3087
weighted avg       0.85      0.86      0.84      3087

classification report of Pack
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2476
           1       0.94      0.93      0.93       611

    accuracy                           0.97      3087
   macro avg       0.96      0.96      0.96      3087
weighted avg       0.97      0.97      0.97      3087

classification report of Serve
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      2442
           1       0.84      0.80      0.82       645

    accuracy                     